# DNN approach for maRAPID

Goodsol Lee, Netlab, Seoul National University

## Get datasets

In [2]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range
import os

#configuration for gpu usage
conf = tf.ConfigProto()
# you can modify below as you want
#conf.gpu_options.per_process_gpu_memory_fraction = 0.4
#conf.gpu_options.allow_growth = True
os.environ['CUDA_VISIBLE_DEVICES']='0'
print(tf.__version__)

1.12.0


<font size="5">Sample training data, validation data, test data from raw data</font>

In [3]:
node_num = 155
num_RSSI_sample = 10

data_set = np.empty((num_RSSI_sample))
channel_label_set = np.empty((1))
TA_label_set = np.empty((1))

for node_index in range(1,node_num+1):
    print('Get sample from node'+str(node_index))
    file_name = 'data/node-'+str(node_index)+'.txt'
    previous_cellId = -1
    cell_sample = list()
    cell_sample_set = list()
    
    with open(file_name, 'r') as f:
        while 1:
            line = f.readline()
            if not line: break
            parsed_line =line.split(' ')
            
            rssi = parsed_line[9]
            channel_state = parsed_line[8]
            current_cellId = parsed_line[10]
            TA_region = parsed_line[11]
            
            if previous_cellId == current_cellId:
                cell_sample.append([rssi, channel_state, TA_region])
            else:
                previous_cellId = current_cellId
                if cell_sample:
                    cell_sample_set.append(cell_sample)
                    cell_sample = list()
        
        cell_sample_set.append(cell_sample)
        sample_length = 0
        for i in range(len(cell_sample_set)):
            sample_length += len(cell_sample_set[i])-num_RSSI_sample+1
        
        print(sample_length)
        
        temp_data_set = np.zeros((sample_length,num_RSSI_sample))
        temp_channel_label_set = np.zeros((sample_length,1))
        temp_TA_label_set = np.zeros((sample_length,1))
        
        for i in range(len(cell_sample_set)):
            cell_sample = cell_sample_set[i]
            cell_sample = np.array(cell_sample)
            for j in range(len(cell_sample_set[i])-num_RSSI_sample+1):
                temp_data_set[j] = cell_sample[j:j+num_RSSI_sample,0]
                temp_channel_label_set[j] = cell_sample[j+num_RSSI_sample-1,1]
                temp_TA_label_set[j] = cell_sample[j+num_RSSI_sample-1,2] 
              
    if node_index == 1:
        data_set = temp_data_set
        channel_label_set = temp_channel_label_set
        TA_label_set = temp_TA_label_set
    else:
        data_set = np.append(data_set,temp_data_set,axis = 0)
        channel_label_set = np.append(channel_label_set,temp_channel_label_set,axis = 0)
        TA_label_set = np.append(TA_label_set,temp_TA_label_set,axis=0)
    
print('Sampling is completed, sample length: ',data_set.shape[0])

Get sample from node1
25753
Get sample from node2
15583
Get sample from node3
25360
Get sample from node4
18582
Get sample from node5
26572
Get sample from node6
21183
Get sample from node7
30573
Get sample from node8
52965
Get sample from node9
26581
Get sample from node10
21779
Get sample from node11
21552
Get sample from node12
16762
Get sample from node13
33371
Get sample from node14
23378
Get sample from node15
52162
Get sample from node16
34773
Get sample from node17
26372
Get sample from node18
29170
Get sample from node19
22372
Get sample from node20
24572
Get sample from node21
41962
Get sample from node22
40942
Get sample from node23
11791
Get sample from node24
23972
Get sample from node25
27971
Get sample from node26
42366
Get sample from node27
29372
Get sample from node28
20782
Get sample from node29
24181
Get sample from node30
25184
Get sample from node31
35176
Get sample from node32
34172
Get sample from node33
25181
Get sample from node34
20571
Get sample from node35


In [4]:
for i in range(1):
    print('me'+'me')

meme


In [5]:
import random

data_length = data_set.shape[0]
valid_num = 100000
test_num = 100000

#get test set
test_index = random.sample(range(0,data_length-test_num),test_num)
test_data = data_set[test_index]
test_channel_label = channel_label_set[test_index]
test_TA_label = TA_label_set[test_index]

#get training set/validation set
train_data_set = np.delete(data_set,test_index,axis=0)
train_channel_label_set = np.delete(channel_label_set,test_index,axis=0)
train_TA_label_set = np.delete(TA_label_set,test_index,axis=0)
data_length = train_data_set.shape[0]

valid_index = random.sample(range(0,data_length),valid_num)

training_data = train_data_set
training_channel_label = train_channel_label_set
training_TA_label = train_TA_label_set

valid_data = train_data_set[valid_index]
valid_channel_label = train_channel_label_set[valid_index]
valid_TA_label = train_TA_label_set[valid_index]

save_data ={
    'training_data':training_data,
    'training_channel_label':training_channel_label,
    'training_TA_label':training_TA_label,
    
    'valid_data':valid_data,
    'valid_channel_label':valid_channel_label,
    'valid_TA_label':valid_TA_label,
    
    'test_data':test_data,
    'test_channel_label':test_channel_label,
    'test_TA_label':test_TA_label
}

In [6]:
# get name of the data path
data_path = 'data/save_data'+str(num_RSSI_sample)+'.pickle'

In [7]:
# save data
with open(data_path,'wb') as f:
    pickle.dump(save_data,f,pickle.HIGHEST_PROTOCOL)

In [8]:
# restore data
with open(data_path,'rb') as f:
    save = pickle.load(f)
    
    train_dataset = save['training_data']
    train_channel_labels = save['training_channel_label']
    train_labels = save['training_TA_label']
    
    valid_dataset = save['valid_data']
    valid_channel_labels = save['valid_channel_label']
    valid_labels = save['valid_TA_label']
    
    test_dataset = save['test_data']
    test_channel_labels = save['test_channel_label']
    test_labels = save['test_TA_label']
    
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (3959538, 10) (3959538, 1)
Validation set (100000, 10) (100000, 1)
Test set (100000, 10) (100000, 1)


In [9]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, num_RSSI_sample)).astype(np.float32)
    # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
    enc.fit(labels)
    labels = enc.transform(labels).toarray()
    return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

/home/gslee/anaconda3/envs/deep-learning-19/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


Training set (3959538, 10) (3959538, 4)
Validation set (100000, 10) (100000, 4)
Test set (100000, 10) (100000, 4)


/home/gslee/anaconda3/envs/deep-learning-19/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/home/gslee/anaconda3/envs/deep-learning-19/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this wa

In [10]:
sample_size = num_RSSI_sample
num_labels = 4

layer1_output_num = 200
layer2_output_num = 200
layer3_output_num = 1024
layer4_output_num = 512
layer5_output_num = 256
num_steps = 1000
batch_size = 100000

graph_gs=tf.Graph()
with graph_gs.as_default():
    tf_dataset_gs=tf.placeholder(tf.float32, shape=(None, sample_size))
    tf_labels_gs=tf.placeholder(tf.float32, shape=(None, num_labels))
    keep_prob = tf.placeholder(tf.float32, shape=(None))
    
    lambda_reg = 0.1
        
    #Regularization
    regularizer = tf.contrib.layers.l2_regularizer(scale=lambda_reg)
    initializer = tf.contrib.layers.variance_scaling_initializer(dtype=tf.float32)
    #neural network consists of two lines
    dense1 = tf.layers.dense(tf_dataset_gs, layer1_output_num, activation=tf.nn.relu, kernel_regularizer=regularizer)
    dense1 = tf.nn.dropout(dense1, keep_prob = keep_prob)
    dense2 = tf.layers.dense(dense1,layer2_output_num, activation=tf.nn.relu, kernel_regularizer=regularizer)
    dense2 = tf.nn.dropout(dense2, keep_prob = keep_prob)
    dense3 = tf.layers.dense(dense2 ,layer3_output_num, activation=tf.nn.relu, kernel_regularizer=regularizer)
    dense3 = tf.nn.dropout(dense3, keep_prob = keep_prob)
    dense4 = tf.layers.dense(dense3 ,layer4_output_num, activation=tf.nn.relu, kernel_regularizer=regularizer)
    dense4 = tf.nn.dropout(dense4, keep_prob = keep_prob)
    dense5 = tf.layers.dense(dense4 ,layer4_output_num, activation=tf.nn.relu, kernel_regularizer=regularizer)
    dense5 = tf.nn.dropout(dense5, keep_prob = keep_prob)
    logits_gs = tf.layers.dense(dense2, num_labels, activation=None)
    
    #Loss
    loss_gs = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf_labels_gs, logits=logits_gs))
    
    global_step = tf.Variable(0, trainable=False)
    starter_learning_rate = 0.005
    
    decay_num =  50*(int)(train_labels.shape[0]/batch_size)
    learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step, decay_num, 1, staircase=True)
    # Optimizer
    optimizer_gs = tf.train.AdamOptimizer(learning_rate).minimize(loss_gs, global_step=global_step)
    
    #Predictions for the training
    prediction_gs = tf.nn.softmax(logits_gs)

In [ ]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.equal(np.argmax(predictions, 1), np.argmax(labels, 1)))
          / predictions.shape[0])

tf.reset_default_graph()
batch_num = (int)(train_labels.shape[0]/batch_size)

with tf.Session(graph=graph_gs, config=conf) as session_gs:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        random = np.random.permutation(train_labels.shape[0])
        loss = 0
        for b in range(batch_num):
            batch_data = train_dataset[random[b*batch_size:(b+1)*batch_size]]
            batch_labels = train_labels[random[b*batch_size:(b+1)*batch_size]].astype(float)
            feed_dict_gs = {tf_dataset_gs: batch_data, tf_labels_gs: batch_labels, keep_prob:1}
            _, l_gs, predictions_l = session_gs.run([optimizer_gs, loss_gs, prediction_gs], feed_dict=feed_dict_gs)
            loss += l_gs
        print("Minibatch loss at step %d: %f" % (step, loss))
        feed_dict_val_gs = {tf_dataset_gs: valid_dataset}
        valid_prediction_gs = session_gs.run(prediction_gs, feed_dict={tf_dataset_gs: valid_dataset, tf_labels_gs: valid_labels, keep_prob:1})
        print("Validation accuracy: %.1f%%" % accuracy(valid_prediction_gs, valid_labels))
        
    feed_dict_test_gs = {tf_dataset_gs: test_dataset, keep_prob:1.0}
    test_prediction_gs = session_gs.run(prediction_gs, feed_dict=feed_dict_test_gs)
    print("Test accuracy: %.1f%%" % accuracy(test_prediction_gs, test_labels))
    saver = tf.train.Saver()
    saver.save(session_gs, "./model_checkpoints/centralized_sample"+str(sample_size))

Initialized
Minibatch loss at step 0: 307.021542
Validation accuracy: 59.9%
Minibatch loss at step 1: 27.795916
Validation accuracy: 65.4%
Minibatch loss at step 2: 25.987112
Validation accuracy: 66.0%
Minibatch loss at step 3: 25.551105
Validation accuracy: 66.5%
Minibatch loss at step 4: 25.283938
Validation accuracy: 66.9%
Minibatch loss at step 5: 25.073948
Validation accuracy: 67.4%
Minibatch loss at step 6: 24.885525
Validation accuracy: 67.6%
Minibatch loss at step 7: 24.700439
Validation accuracy: 67.9%
Minibatch loss at step 8: 24.530154
Validation accuracy: 68.3%
Minibatch loss at step 9: 24.329343
Validation accuracy: 68.5%
Minibatch loss at step 10: 24.120857
Validation accuracy: 68.7%
Minibatch loss at step 11: 23.954716
Validation accuracy: 69.6%
Minibatch loss at step 12: 23.899897
Validation accuracy: 64.9%
Minibatch loss at step 13: 24.052851
Validation accuracy: 65.7%
Minibatch loss at step 14: 23.462222
Validation accuracy: 71.3%
Minibatch loss at step 15: 22.958670


Minibatch loss at step 128: 11.469527
Validation accuracy: 86.5%
Minibatch loss at step 129: 11.723852
Validation accuracy: 85.0%
Minibatch loss at step 130: 11.379695
Validation accuracy: 86.3%
Minibatch loss at step 131: 11.849954
Validation accuracy: 86.2%
Minibatch loss at step 132: 11.484133
Validation accuracy: 86.5%
Minibatch loss at step 133: 11.190518
Validation accuracy: 86.0%
Minibatch loss at step 134: 11.427199
Validation accuracy: 83.8%
Minibatch loss at step 135: 11.766766
Validation accuracy: 86.5%
Minibatch loss at step 136: 11.241841
Validation accuracy: 86.0%
Minibatch loss at step 137: 11.639317
Validation accuracy: 86.5%
Minibatch loss at step 138: 11.570413
Validation accuracy: 86.6%
Minibatch loss at step 139: 11.343943
Validation accuracy: 85.4%
Minibatch loss at step 140: 11.469617
Validation accuracy: 86.6%
Minibatch loss at step 141: 11.260540
Validation accuracy: 86.0%
Minibatch loss at step 142: 11.704328
Validation accuracy: 86.2%
Minibatch loss at step 14

Minibatch loss at step 255: 11.182338
Validation accuracy: 86.2%
Minibatch loss at step 256: 11.308809
Validation accuracy: 86.7%
Minibatch loss at step 257: 11.061762
Validation accuracy: 86.0%
Minibatch loss at step 258: 11.140257
Validation accuracy: 86.8%
Minibatch loss at step 259: 11.312326
Validation accuracy: 86.1%
Minibatch loss at step 260: 11.064257
Validation accuracy: 86.8%
Minibatch loss at step 261: 11.047774
Validation accuracy: 86.8%
Minibatch loss at step 262: 11.214629
Validation accuracy: 86.6%
Minibatch loss at step 263: 11.099786
Validation accuracy: 86.5%
Minibatch loss at step 264: 11.209942
Validation accuracy: 84.9%
Minibatch loss at step 265: 11.136719
Validation accuracy: 86.8%
Minibatch loss at step 266: 11.004724
Validation accuracy: 86.5%
Minibatch loss at step 267: 11.255009
Validation accuracy: 85.4%
Minibatch loss at step 268: 11.094289
Validation accuracy: 86.8%
Minibatch loss at step 269: 11.056015
Validation accuracy: 86.5%
Minibatch loss at step 27

In [93]:
with tf.Session(graph=graph_gs, config=conf) as session_gs:
    tf.global_variables_initializer().run()
    feed_dict_test_gs = {tf_dataset_gs: test_dataset, keep_prob:1}
    test_prediction_gs = session_gs.run(prediction_gs, feed_dict=feed_dict_test_gs)
    print("Test accuracy: %.1f%%" % accuracy(test_prediction_gs, test_labels))

Test accuracy: 52.5%
